In [1]:
#configure plotting
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
matplotlib.rcParams['figure.figsize'] = (16, 10)
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.size'] = 25
matplotlib.rcParams['font.family'] = 'serif'

import GPy
import numpy as np
from matplotlib import pyplot as plt

np.random.seed(42)

from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import accuracy_score

import config
import Data

import ScoreFunctions
import Utils

# Initialize experiment
- Train dataset size changes from start_n to end_n  
- Info about the experiment is saved at ./Results/name  
- draw_* options could be True only if ndim <= 2

In [2]:
info = config.ExperimentInfo(
    ndim_    = 3, 
    total_n_ = 1000, 
    start_n_ = 50, 
    end_n_   = 500, 
    test_n_  = 500, 
    name_    = "SkinTime2",
    dataset_     = Data.Skin,
    draw_data_   = False,
    draw_scores_ = False,
    update_freq_ = 10,
    comparsion_  = False)

In [3]:
info.dataset.X

array([[ 74,  85, 123],
       [ 73,  84, 122],
       [ 72,  83, 121],
       ...,
       [163, 162, 112],
       [163, 162, 112],
       [255, 255, 255]])

There are some needed variables. We aren't interested in them since we look into active learning part, not model charachteristics. Still the variables are needed to make it all work.

In [4]:
var = 100
lengthscale = 1

k = GPy.kern.RBF(info.ndim, variance = var, lengthscale = lengthscale)
lik = GPy.likelihoods.Bernoulli()

# An example Inference
To see whether gaussian process learns on the data

In [5]:
m = GPy.core.GP(
    X      = info.dataset.X_train,
    Y      = info.dataset.y_train.reshape(-1, 1), 
    kernel = k, 
    inference_method = GPy.inference.latent_function_inference.expectation_propagation.EP(),
    likelihood = lik)

m.optimize()
print(m.kern)

if (info.draw_data):
    m.plot(plot_density = True)
    
prediction = m.predict(info.dataset.X_test.reshape(-1, info.ndim))[:][0]
print(accuracy_score(info.dataset.y_test, Utils.to_labels(prediction)))

  rbf.         |               value  |  constraints  |  priors
  variance     |  142.53276680389752  |      +ve      |        
  lengthscale  |  116.48880124747016  |      +ve      |        
0.952


# Active learning part

In [6]:
# this only needed if we want to compare active learning with traditional approaches

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

acc_lr = []
acc_kn = []
acc_dt = []
acc_rf = []

# The main loop
Train 6 gps with different score function

In [7]:
models = []

X_train_0   = info.dataset.X_train
y_train_0   = info.dataset.y_train
U_0         = info.dataset.U
y_U_0       = info.dataset.y_U

score_functions = [
    ScoreFunctions.calculate_scores_mvar,
    ScoreFunctions.calculate_scores_Hvar, 
    ScoreFunctions.calculate_scores_RKHS,
    ScoreFunctions.calculate_scores_rand, 
    ScoreFunctions.calculate_scores_l2fm,
    ScoreFunctions.calculate_scores_sqsm]

import time

for score in score_functions:
    info.dataset.X_train = X_train_0
    info.dataset.y_train = y_train_0
    info.dataset.U = U_0
    info.dataset.y_U = y_U_0
    
    k = GPy.kern.RBF(info.ndim, variance = var, lengthscale = lengthscale)
    m = GPy.core.GP(
        X      = info.dataset.X_train,
        Y      = info.dataset.y_train.reshape(-1, 1), 
        kernel = k, 
        inference_method = GPy.inference.latent_function_inference.expectation_propagation.EP(),
        likelihood = lik)
    
    K     = Utils.get_K(m, info.dataset.X_train)
    inv_K = Utils.get_inv_K(m, info.dataset.X_train) 
    
    score_name = str(score)[27:31]
    print(score_name)
    
    start = time.time()
    score_time = 0
    
    for i in tqdm(range(info.start_n, info.end_n)):
        # 0. learn GP on the data with update_frequency
        if i % info.update_freq == 0:
            m = GPy.core.GP(
                X      = info.dataset.X_train,
                Y      = info.dataset.y_train.reshape(-1, 1), 
                kernel = m.kern, 
                inference_method = GPy.inference.latent_function_inference.expectation_propagation.EP(),
                likelihood = lik)

            m.optimize(messages = False)
        
        # 1. calculate score at each point of the rest dataset in U and y_u, get the index of maximum score
        start1 = time.time()
        scores = score(np.array(info.dataset.U), m, info.dataset.X_train, info.dataset.y_train, inv_K)
        end1 = time.time()
        
        score_time += end1 - start1
        ind = np.argmax(scores)
        
        # 2. append new point to the training dataset and update inv_K
        info.dataset.X_train = np.concatenate(
            (info.dataset.X_train, 
             info.dataset.U[ind].reshape(-1, info.ndim)), 
            axis = 0)
        info.dataset.y_train = np.append(info.dataset.y_train, info.dataset.y_U[ind]).reshape(-1, 1)
        
        a = m.kern.K(
            info.dataset.U[ind].reshape(-1, info.ndim), 
            info.dataset.X_train[:info.dataset.X_train.shape[0]-1]).T
        
        inv_K = Utils.update_inv_K(m, info.dataset.X_train, inv_K, info.dataset.U[ind], a)
        
        # 2.5. draw score if needed
        # placing this function call right here is important
        # since draw_score should see whole U included the point to be deleted
        if info.draw_scores:
            info.dataset.draw_score(scores, score_name, info.name, i, m, score, inv_K)

        # 3. delete new point from U since it is labeled now
        info.dataset.U = np.delete(info.dataset.U, ind, axis = 0)
        info.dataset.y_U = np.delete(info.dataset.y_U, ind)

        # 4. count accuracy
        prediction = m.predict(info.dataset.X_test.reshape(-1, info.ndim))[:][0]
        cur_accuracy = accuracy_score(info.dataset.y_test, Utils.to_labels(prediction))
        
        info.per_score_acc[score_name].append(cur_accuracy)
        info.save(score_name)
        
        del a    
        del scores
        del prediction
        
        # 5. compare with other models
        # it is important to learn traditional methods on train dataset of rand score function
        if info.comparsion and score == ScoreFunctions.calculate_scores_rand:
            lr = LogisticRegression(solver = 'lbfgs').fit(info.dataset.X_train, info.dataset.y_train)
            acc_lr.append(lr.score(info.dataset.X_test, info.dataset.y_test))

            kn = KNeighborsClassifier().fit(info.dataset.X_train, info.dataset.y_train)
            acc_kn.append(kn.score(info.dataset.X_test, info.dataset.y_test))

            dt = DecisionTreeClassifier().fit(info.dataset.X_train, info.dataset.y_train)
            acc_dt.append(dt.score(info.dataset.X_test, info.dataset.y_test))

            rf = RandomForestClassifier().fit(info.dataset.X_train, info.dataset.y_train)
            acc_rf.append(rf.score(info.dataset.X_test, info.dataset.y_test))

            del lr
            del kn
            del dt
            del rf
        
    end = time.time()
    info.times[score_name] = score_time
    print(end - start)
    print(score_time)
    #after the gp is learned save it to models
    models.append(m)

mvar


 /home/daria.kotova/anaconda2/lib/python2.7/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1



34.3799080849
0.565038919449
Hvar



37.5859041214
1.06514453888
RKHS



38.0880661011
1.16137552261
rand



37.4343550205
0.0121328830719
l2fm



39.0847480297
2.16635560989
sqsm


KeyboardInterrupt: 

In [ ]:
out = open("./Results/" + info.name + "/description.txt", 'a')
        
out.write(
      "time rand = " + str(info.times["rand"]) + "\n"
    + "time mvar = " + str(info.times["mvar"]) + "\n"
    + "time RKHS = " + str(info.times["RKHS"]) + "\n"
    + "time Hvar = " + str(info.times["Hvar"]) + "\n"
    + "time l2fm = " + str(info.times["l2fm"]) + "\n")

out.close()

# Analysis
- Plot gaussian processes if possible to see if there are alive  
- Plot accuracies (size of training dataset). Sqsm is exluded since we do not conduct experiments for this criterion

In [ ]:
for m in models:
    print(m.kern)
    if info.draw_data:
        m.plot(plot_density = True)

In [ ]:
acc_rand = np.loadtxt("./Results/" + info.name + "/accuracies/acc_rand.txt")
acc_mvar = np.loadtxt("./Results/" + info.name + "/accuracies/acc_mvar.txt")
acc_RKHS = np.loadtxt("./Results/" + info.name + "/accuracies/acc_RKHS.txt")
acc_Hvar = np.loadtxt("./Results/" + info.name + "/accuracies/acc_Hvar.txt")
acc_l2fm = np.loadtxt("./Results/" + info.name + "/accuracies/acc_l2fm.txt")
# acc_sqsm = np.loadtxt("./Results/" + info.name + "/accuracies/acc_sqsm.txt")

In [ ]:
def Range(start, length, partition):
    return range(start, start + length // partition)

In [ ]:
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
matplotlib.rcParams['figure.figsize'] = (16, 10)
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.size'] = 25
matplotlib.rcParams['font.family'] = 'serif'

# first 1/partition of acc is shown
partition = 2

plt.plot(Range(info.start_n, len(acc_rand), partition), acc_rand[:len(acc_rand)//partition], label='rand')
plt.plot(Range(info.start_n, len(acc_mvar), partition), acc_mvar[:len(acc_mvar)//partition], label='var')
# plt.plot(Range(info.start_n, len(acc_sqsm), partition), acc_sqsm[:len(acc_sqsm)//partition], label='L2')
plt.plot(Range(info.start_n, len(acc_RKHS), partition), acc_RKHS[:len(acc_RKHS)//partition], label='RKHS')
plt.plot(Range(info.start_n, len(acc_Hvar), partition), acc_Hvar[:len(acc_Hvar)//partition], label='Hvar')
plt.plot(Range(info.start_n, len(acc_l2fm), partition), acc_l2fm[:len(acc_l2fm)//partition], label='l2fm')

# if info.comparsion:
#     plt.plot(Range(info.start_n, len(acc_lr), partition), acc_lr[:len(acc_lr)//partition], label='log. regression')
#     plt.plot(Range(info.start_n, len(acc_kn), partition), acc_kn[:len(acc_kn)//partition], label='k-nearest')
#     plt.plot(Range(info.start_n, len(acc_dt), partition), acc_dt[:len(acc_dt)//partition], label='decision tree')
#     plt.plot(Range(info.start_n, len(acc_rf), partition), acc_rf[:len(acc_rf)//partition], label='random forest')

plt.title("accuracy(n)")
plt.ylabel("accuracy")
plt.xlabel("size of training dataset")
plt.legend()
plt.savefig("./Results/" + info.name + "/accuracyGP" + str(partition) + ".png")